In [19]:
# scripts/daily_trade_suggestions.py

import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime
import json
from pathlib import Path

DATA_RECOMENDATIONS = Path("data/recommendations")
DATA_SUGGESTIONS = Path("data/suggestions")

for p in (DATA_RECOMENDATIONS, DATA_SUGGESTIONS):
    p.mkdir(parents=True, exist_ok=True)


In [20]:
def latest_file_in_directory(directory=DATA_RECOMENDATIONS):
    latest_file = max(
        f for f in os.listdir(directory)
        if f.startswith("trade_suggestions_") and f.endswith(".csv")
    )
    return latest_file

In [21]:
latest_file = latest_file_in_directory(DATA_RECOMENDATIONS)
df_all = pd.read_csv(os.path.join(DATA_RECOMENDATIONS, latest_file))
df_all.shape

(54, 10)

In [22]:
df_all = df_all.rename(columns={"Unnamed: 0": "ticker"})
df_all.head()

,ticker,price,ma50,mean_ret_fwd_1d,mean_ret_fwd_5d,mean_ret_fwd_20d,appearances,appearances_persistent,avg_HotScore,rule_score
0,RGC,32.290001,22.341200,2.736519,13.355257,75.937801,28,329.0,0.851327,6
1,SNDK,527.630005,292.379200,2.583637,12.714612,73.378413,26,292.0,0.824386,6
2,RKLB,88.570000,66.014100,0.882234,8.937528,40.555221,27,365.0,0.758954,6
3,MU,435.279999,287.940124,2.314307,8.475579,40.217608,28,315.0,0.799693,6
4,BE,165.389999,111.132600,1.842494,7.904877,10.022235,22,270.0,0.789403,6


In [17]:
agg_map = {
    "price": "mean",
    "ma50": "mean",
    "mean_ret_fwd_1d": "mean",
    "mean_ret_fwd_5d": "mean",
    "mean_ret_fwd_20d": "mean",
    "appearances": "sum",
    "appearances_persistent": "max",
    "avg_HotScore": "mean",
    "rule_score": "max"
}

df_outcome = (
    df_all
    .groupby("ticker", as_index=False)
    .agg(agg_map)
)

df_outcome.sort_values("avg_HotScore", ascending=False).head()

,ticker,price,ma50,mean_ret_fwd_1d,mean_ret_fwd_5d,mean_ret_fwd_20d,appearances,appearances_persistent,avg_HotScore,rule_score
331,RGC,31.128044,20.542077,3.431695,17.173441,94.598242,3005.0,329.0,0.849555,6
416,VERA,46.847829,41.670729,0.446763,7.664952,21.450928,1872.0,248.0,0.823859,6
361,SNDK,444.220096,265.861961,2.048670,13.220684,56.712541,2477.0,292.0,0.823007,6
238,LITE,347.583985,323.500581,0.346679,3.339552,9.330412,2274.0,287.0,0.796931,6
268,MU,373.350515,270.401322,2.020016,7.993496,36.565373,2734.0,315.0,0.796165,6


In [18]:
today = datetime.now().strftime("%Y%m%d")

out_file = DATA_SUGGESTIONS / f"trade_suggestions_{today}.csv"

df_outcome.to_csv(out_file, index=False)
